In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa

In [2]:
directory = 'C:\datasets\img_align_celeba\img_align_celeba'

In [3]:
tag = pd.read_csv('C:/datasets/list_attr_celeba.csv')

In [4]:
col = ['image_id', 'Male', 'Mustache', 'Bald', 'Smiling', 'Young']
tag = tag[col]
tag = tag.replace(-1, 0)
col = col[1:]
tag

,image_id,Male,Mustache,Bald,Smiling,Young
0,000001.jpg,0,0,0,1,1
1,000002.jpg,0,0,0,1,1
2,000003.jpg,1,0,0,0,1
3,000004.jpg,0,0,0,0,1
4,000005.jpg,0,0,0,0,1
...,...,...,...,...,...,...
202594,202595.jpg,0,0,0,0,1
202595,202596.jpg,1,0,0,1,1
202596,202597.jpg,1,0,0,1,1
202597,202598.jpg,0,0,0,1,1


In [5]:
train = tag.sample(frac = .8, random_state=42)
test = tag.drop(train.index)
train

,image_id,Male,Mustache,Bald,Smiling,Young
93241,093242.jpg,1,0,0,0,1
121413,121414.jpg,0,0,0,0,1
10624,010625.jpg,1,0,0,0,1
82523,082524.jpg,0,0,0,1,1
73192,073193.jpg,0,0,0,1,0
...,...,...,...,...,...,...
36890,036891.jpg,1,0,0,0,1
1992,001993.jpg,0,0,0,0,1
47852,047853.jpg,0,0,0,1,1
79821,079822.jpg,0,0,0,1,1


In [6]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [7]:
train_generator=datagen.flow_from_dataframe(
dataframe=train,
directory = directory,
x_col="image_id",
y_col=col,
subset="training",
batch_size=16,
seed=42,
shuffle=True,
    class_mode='raw',
target_size=(256,256))

valid_generator=datagen.flow_from_dataframe(
dataframe=train,
directory=directory,
x_col="image_id",
y_col=col,
subset="validation",
batch_size=16,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(256,256))

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test,
directory=directory,
x_col="image_id",
y_col=col,
batch_size=16,
seed=42,
shuffle=False,
class_mode='raw',
target_size=(256,256))

Found 121560 validated image filenames.
Found 40519 validated image filenames.
Found 40520 validated image filenames.


In [17]:
class Affine(tf.keras.layers.Layer):
    def __init__(self, n_channels:int, biased:bool):
        super(Affine, self).__init__()
        self.n_channels = n_channels
        self.biased = biased
        
        self.a = tf.Variable(tf.ones([1,1,self.n_channels]), dtype = 'float32') if self.biased else tf.Variable(tf.random.normal([1,1,self.n_channels], 0, 0.001), dtype = 'float32')
        self.b = tf.Variable(tf.zeros([1,1,self.n_channels]), dtype = 'float32') if self.biased else None
    
    def call(self, X):
        X = X * self.a
        if self.biased:
            X = X + self.b
        return X
        
class ResLayer(tf.keras.layers.Layer):
    def __init__(self, n_dims:int, n_patches:int):
        super(ResLayer, self).__init__()
        self.n_dims = n_dims
        self.n_patches = n_patches
        
        self.affine_pre1 = Affine(self.n_dims, True)
        self.affine_pre2 = Affine(self.n_dims, True)
        self.affine_pos1 = Affine(self.n_dims, False)
        self.affine_pos2 = Affine(self.n_dims, False)
        self.t = tf.keras.layers.Permute((2,1))
        self.linear1 = tf.keras.layers.Dense(self.n_patches, kernel_initializer = tf.keras.initializers.glorot_normal())
        self.linear2 = tf.keras.layers.Dense(self.n_dims*4, kernel_initializer = tf.keras.initializers.glorot_normal())
        self.linear3 = tf.keras.layers.Dense(self.n_dims, kernel_initializer = tf.keras.initializers.glorot_normal())
        
    def call(self, X):
        #Cross-patch
        y = self.affine_pre1(X)
        y = self.t(y)
        y = self.linear1(y)
        y = self.t(y)
        y = self.affine_pos1(y)
        X = X + y
        #Cross-channel
        y = self.affine_pre2(X)
        y = self.linear2(y)
        y = tf.nn.gelu(y)
        y = self.linear3(y)
        y = self.affine_pos2(y)
        y = X + y
        return y
    

class ResMLP(tf.keras.models.Model):
    def __init__(self, n_dims:int, image_size:int, patch_size:int, n_res_layers:int, n_labels:int, mode:str = 'softmax'):
        super(ResMLP, self).__init__()
        self.n_dims = n_dims
        self.image_size = image_size
        self.patch_size = patch_size
        if (self.image_size % self.patch_size) != 0:
            raise ValueError('size error')
        self.n_patches = int((tf.square(self.image_size) / tf.square(self.patch_size)).numpy())
        self.n_res_layers = n_res_layers
        self.n_labels = n_labels
        if mode not in ['sigmoid','softmax']:
            raise ValueError('mode must be sigmoid or softmax')
        else:
            self.mode = mode
        
        self.patchConv = tf.keras.layers.Conv2D(self.n_dims, (self.patch_size, self.patch_size), strides = (self.patch_size, self.patch_size))
        self.reshapeL = tf.keras.layers.Reshape((self.n_patches, self.n_dims))
        self.ResLayers = [ResLayer(self.n_dims, self.n_patches) for x in range(self.n_res_layers)]
        self.gap = tf.keras.layers.GlobalAveragePooling1D()
        self.classifier = tf.keras.layers.Dense(self.n_labels if self.n_labels > 2 else 1, activation = mode, kernel_initializer = tf.keras.initializers.glorot_normal(seed = 42))
        
    def call(self, X):
        X = self.patchConv(X)
        X = self.reshapeL(X)
        for reslayer in self.ResLayers:
            X = reslayer(X)
        X = self.gap(X)
        X = self.classifier(X)
        return X

In [18]:
resmlp = ResMLP(384, 256, 16, 12, len(col), mode = 'sigmoid')
opt = tf.keras.optimizers.Adam(.00005)
resmlp.compile(optimizer=opt, loss=tfa.losses.focal_loss.sigmoid_focal_crossentropy, metrics='acc')

In [19]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
resmlp.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

Epoch 1/10
7597/7597 [==============================] - 668s 87ms/step - loss: 0.1942 - acc: 0.4437 - val_loss: 0.1402 - val_acc: 0.7283
Epoch 2/10
7597/7597 [==============================] - 664s 87ms/step - loss: 0.1268 - acc: 0.7393 - val_loss: 0.1213 - val_acc: 0.7938
Epoch 3/10
7597/7597 [==============================] - 667s 88ms/step - loss: 0.1140 - acc: 0.7694 - val_loss: 0.1099 - val_acc: 0.7903
Epoch 4/10
7597/7597 [==============================] - 680s 90ms/step - loss: 0.1070 - acc: 0.7832 - val_loss: 0.1051 - val_acc: 0.8060
Epoch 5/10
7597/7597 [==============================] - 699s 92ms/step - loss: 0.1014 - acc: 0.7917 - val_loss: 0.1035 - val_acc: 0.8280
Epoch 6/10
7597/7597 [==============================] - 858s 113ms/step - loss: 0.0967 - acc: 0.7986 - val_loss: 0.0967 - val_acc: 0.8028
Epoch 7/10
7597/7597 [==============================] - 1023s 135ms/step - loss: 0.0925 - acc: 0.8049 - val_loss: 0.0946 - val_acc: 0.8187
Epoch 8/10
7597/7597 [================